In [2]:
import os
import random
import numpy as np
from PIL import Image
import imgaug.augmenters as iaa

#Uncomment lower line if you using numpy latest version or it will give error
np.bool = np.bool_

def save_augmented_images(folder_path, images, augmenter, target):
    current_count = len(images)
    image_index = 0  # To ensure unique filenames

    while current_count < target:
        try:
            # Randomly choose an image
            img_path = random.choice(images)
            img = Image.open(img_path).convert("RGB")  # Ensure RGB mode
            img_array = np.array(img)

            # Apply augmentation
            augmented_image = augmenter(image=img_array)
            augmented_image = Image.fromarray(augmented_image)

            # Generate a unique filename
            new_filename = os.path.join(folder_path, f"aug_{image_index}.jpg")
            while os.path.exists(new_filename):  # Ensure uniqueness
                image_index += 1
                new_filename = os.path.join(folder_path, f"aug_{image_index}.jpg")

            # Save the augmented image
            augmented_image.save(new_filename)

            current_count += 1
            image_index += 1  # Update index for next iteration
            
        except Exception as e:
            print(f"Error processing image {img_path}: {e}")

def augment_data(path, size):
    dataset_path = path
    target_count = size  

    augmenter = iaa.Sequential([
        iaa.SomeOf((2, 4), [  # Apply 2 to up to 4 augmentations 
            iaa.Sharpen(alpha=(0, 0.5), lightness=(0.75, 1.25)),  #  sharpening
            iaa.Crop(percent=(0, 0.1)),  # Less cropping
            iaa.Sometimes(0.5, iaa.Affine(rotate=(-10, 10))),  #  probability based  rotation
            iaa.Sometimes(0.3, iaa.GaussianBlur(sigma=(0, 0.3))),  #  blurring
            iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.01*255), per_channel=0.5),  #  noise
            iaa.Sometimes(0.5, iaa.ContrastNormalization((0.9, 1.1))),  # Contrast adjustment
        ])
    ], random_order=True)  # Apply in random order

    # Detect folders in the dataset directory
    folders = [d for d in os.listdir(dataset_path) if os.path.isdir(os.path.join(dataset_path, d))]

    # Augment images for each detected folder
    for folder in folders:
        folder_path = os.path.join(dataset_path, folder)
        images = [os.path.join(folder_path, img) for img in os.listdir(folder_path) if img.endswith(('.png', '.jpg', '.jpeg'))]

        # Perform augmentation if needed
        if len(images) < target_count:
            print(f"Augmenting {folder} from {len(images)} to {target_count} images.")
            save_augmented_images(folder_path, images, augmenter, target_count)
        else:
            print(f"{folder} already has {len(images)} images or more.")

    print("Data augmentation completed!")


In [6]:
augment_data(r"Brain tumor_D3_White Strip Normalization", 5700)

Augmenting glioma_tumor from 5692 to 5700 images.
Augmenting meningioma_tumor from 4519 to 5700 images.
Augmenting no_tumor from 3170 to 5700 images.
Augmenting pituitary_tumor from 4507 to 5700 images.
Data augmentation completed!
